# Wine project pre-processing

In [1]:
import pandas as pd
import requests
import json
import numpy as np
import datetime
import time
import tweepy
import re

from datetime import datetime, timedelta

In [4]:
df = pd.read_csv('..\Data\winemag-data-130k-v2.csv', index_col=0)

## Extract information from title

In [ ]:
titles = df[['title']].values.tolist()
titles = [item for sublist in titles for item in sublist]
titles

In [ ]:
producer = []
years = []
vineyard_grape = []
region = []

for title in titles:
    #print(title)
    result = re.match(r'(\w.+) (\d*) ([^(]+) (\(.*\))', title)
    if result:
        #print(result.groups())
        producer.append(result.group(1))
        years.append(result.group(2))
        vineyard_grape.append(result.group(3))
        region.append(result.group(4)[1:-1])
    else:
        producer.append(0)
        years.append(0)
        vineyard_grape.append(0)
        region.append(0)

df['producer'] = producer
df['year'] = years
df['vineyard_grape'] = vineyard_grape
df['region_from_name'] = region

In [ ]:
df

## Get number of followers of the rating person

In [ ]:
df['taster_twitter_handle']

In [ ]:
# Enter your Twitter API credentials
bearer_token = "<insert bearer token from twitter API>"

In [ ]:
def getFollowerCount(handle):
    client = tweepy.Client(bearer_token)

    # Get the follower count of a Twitter handle
    user_ids = [handle]
    response = client.get_users(usernames=user_ids, user_fields=["public_metrics"])    

    if (response.data):
        for user in response.data:
            return user.public_metrics['followers_count']
    else:
        print('Not found: ' + handle)
        return 0

In [ ]:
handles = df[['taster_twitter_handle']].values.tolist()
handles

In [ ]:
dict = {}
followers = []

for handle in handles:
    value = handle[0]
    print(value)

    try:
        result = re.match(r'@(\w+)', value)
    except:
        print('error during parsing')
        followers.append(0)
        continue;

    if result:
        correct_handle = result.group(1)
        #print(correct_handle)
        if (correct_handle in dict):
            follower_count = dict[correct_handle]
            followers.append(follower_count)
        else:
            follower_count = getFollowerCount(correct_handle)
            dict[correct_handle] = follower_count
            followers.append(follower_count)

df['followers'] = followers

In [ ]:
df

In [ ]:
df.to_csv('..\Data\winemag-data-130k-v2+extract.csv', sep=';')

## Get weather information

Die klimatischen Minimalanforderungen für den Weinbau liegen bei einer mittleren Jahrestemperatur von mindestens 9 Grad Celsius, einer Durchschnittstemperatur des wärmsten Monats von mindestens 18 Grad Celsius, einer maximalen Tiefsttemperatur im Winter von minus 13 Grad Celsius, mindestens 1.300 Sonnenstunden pro Jahr, einer jährlichen Niederschlagsmenge von mindestens 500 mm (in warmem Klima mindestens 750 mm) sowie einer Vegetationsperiode (der Zeit zwischen dem letzten und dem ersten Frost) von mindestens 180 Tagen. Die optimale Temperatur für das Traubenwachstum beträgt zwischen 25 und 28 Grad Celsius. Die Temperatur hängt maßgeblich davon ab, in welcher Höhe die Reben stehen; als Faustregel gilt, dass die Temperatur pro 100 Meter Höhenunterschied aufwärts um 0,6 Grad sinkt.

In [ ]:
apikey = '<insert openweather api key>'

In [ ]:
df = df[['country', 'province', 'region_1']]
df = df[:50000] # max. 50.000 request / day on historic API

df.info()

In [ ]:
def getGeo(url):
    response = requests.get(url)
    data_str = response.json()
    if (response.status_code != 200):
        print(data_str)
        return 0, 0
    elif (len(data_str) > 0):
        for entry in data_str:
            return entry['lat'], entry['lon']
    else:
        return 0, 0

In [ ]:
def getTemp(url):
    response = requests.get(url)
    data_str = response.json()
    if (response.status_code != 200):
        print(data_str)
        return 0, 0, 0
    elif (len(data_str) > 0):
        return (data_str['result']['temp']['median'] - 273.15), (data_str['result']['humidity']['median']), (data_str['result']['wind']['median'])
    else:
        return 0, 0, 0

In [ ]:
dict = {}
lats = []
lons = []

for index, row in df.iterrows():
    country = row['country']
    province = row['province']
    region = row['region_1']

    url = "http://pro.openweathermap.org/geo/1.0/direct?q="
    if (pd.isna(country) is False):
        url += str(country) + ","

    if (pd.isna(province) is False):
        url += str(province) + ","
    
    if (pd.isna(region) is False):
        url += str(region) + ","
        
    if (url[-1] == ','):
        url = url[:-2]

    url += "&limit=1&appid=" + apikey
    
    if (url in dict):
        lat, lon = dict[url]
        #print('got from dict:' + str(lat) + "," + str(lon))
    else:
        lat, lon = getGeo(url)
        dict[url] = (lat, lon)
        #print('got from url:' + str(lat) + "," + str(lon))
    
    lats.append(lat)
    lons.append(lon)

df['lat'] = lats
df['lon'] = lons

In [ ]:
df.to_csv('..\Data\winemag-data-130k-v2+geo.csv', sep=';')

In [ ]:
dict = {}
temps = []
humidities = []
winds = []

for index, row in df.iterrows():
    lat = row['lat']
    lon = row['lon']

    if (lat == 0 and lon == 0):
        temps.append(0)
        humidities.append(0)
        winds.append(0)
        continue

    date_time_start = datetime(2022, 1, 7, 13, 00)
    unix_time_start = time.mktime(date_time_start.timetuple())

    # with our subscription we can only get data from last year.
    url = "https://history.openweathermap.org/data/2.5/aggregated/day?lat="+str(round(lat, 2))+"&lon="+str(round(lon, 2))+"&month=7&day=1&units=metric&appid=" + apikey
    
    if (url in dict):
        temp, humidity, wind = dict[url]
        #print('got from dict:' + str(lat) + "," + str(lon))
    else:
        temp, humidity, wind = getTemp(url)
        dict[url] = (temp, humidity, wind)
        #print('got from url:' + str(lat) + "," + str(lon))
    
    temps.append(temp)
    humidities.append(humidity)
    winds.append(wind)

df['temp'] = temps
df['humidity'] = humidities
df['wind'] = winds

df

In [ ]:
df.to_csv('..\Data\winemag-data-130k-v2+temp.csv', sep=';')

In [ ]:
#df = df = pd.read_csv('..\Data\winemag-data-130k-v2+geo.csv', index_col=0, delimiter=';')
#df

## Extract attributes from description

In [32]:
df = pd.read_csv('..\Data\winemag-data-130k-v2+extract.csv', index_col=0, delimiter=';')
df_words = pd.read_csv('..\Process\wordlist_output.csv', delimiter=';')

In [19]:
df_words.sort_values('total', ascending=False, inplace=True)
df_words.reset_index(drop=True, inplace=True)
df_words

,word,in_documents,total
0,fruit,33,36
1,acidity,29,30
2,black,24,29
3,aromas,27,27
4,cherry,23,26
5,tannins,25,25
6,oak,16,19
7,ripe,18,18
8,white,12,15
9,red,14,14


In [20]:
descriptions = df[['description']].values.tolist()

In [21]:
dict = {}
words = []

for description in descriptions:
    attributes = []
    for word in df_words['word']:
        if word in description[0]:
            attributes.append(word)

    words.append(attributes)

words

[['fruit', 'acidity', 'ripe'],
 ['fruit', 'acidity', 'tannins', 'ripe', 'red'],
 ['acidity'],
 ['aromas', 'dry'],
 [],
 ['fruit', 'acidity', 'aromas', 'dark', 'plum', 'bodied', 'full'],
 ['acidity', 'aromas', 'tannins', 'white', 'red', 'bright'],
 ['acidity', 'spice', 'dry'],
 ['fruit', 'dry'],
 ['fruit', 'acidity', 'spice', 'dry'],
 ['oak', 'plum'],
 ['red', 'dry'],
 ['black', 'cherry', 'oak', 'red', 'rich'],
 ['aromas', 'tannins', 'oak', 'plum', 'dry'],
 ['ripe'],
 ['dry'],
 ['acidity', 'aromas', 'oak', 'red', 'plum'],
 ['fruit', 'black', 'aromas', 'cherry', 'oak', 'dark'],
 ['fruit', 'black', 'aromas', 'oak', 'red', 'spice', 'bodied', 'full'],
 ['fruit', 'aromas', 'cherry', 'ripe', 'plum', 'blend'],
 ['black', 'aromas', 'oak', 'red', 'dark'],
 ['oak'],
 ['fruit', 'acidity', 'aromas', 'white', 'bright'],
 ['acidity', 'black', 'aromas', 'red', 'plum'],
 ['black', 'cherry', 'tannins', 'oak', 'dry'],
 ['black', 'aromas', 'spice', 'bodied', 'full'],
 ['fruit', 'acidity', 'aromas', 'oak',

In [33]:
df['desc'] =  [','.join(ele) for ele in words]
df

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,producer,year,vineyard_grape,region_from_name,followers,desc
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Nicosia,2013.0,Vulkà Bianco,Etna,7542,"fruit,acidity,ripe"
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Quinta dos Avidagos,2011.0,Avidagos Red,Douro,1153,"fruit,acidity,tannins,ripe,red"
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Rainstorm,2013.0,Pinot Gris,Willamette Valley,0,acidity
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,St. Julian,2013.0,Reserve Late Harvest Riesling,Lake Michigan Shore,0,"aromas,dry"
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Sweet Cheeks,2012.0,Vintner's Reserve Wild Child Block Pinot Noir,Willamette Valley,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),Dr. H. Thanisch (Erben Müller-Burggraef),2013.0,Brauneberger Juffer-Sonnenuhr Spätlese Riesling,Mosel,0,"fruit,acidity,sweet"
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation,Citation,2004.0,Pinot Noir,Oregon,0,"fruit,cherry,red,full"
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,Domaine Gresser,2013.0,Kritt Gewurztraminer,Alsace,1153,"fruit,ripe,spice,dry"
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss,Domaine Marcel Deiss,2012.0,Pinot Gris,Alsace,1153,"acidity,spice,dry"


In [34]:
df.to_csv('..\Data\winemag-data-130k-v2+extract.csv', sep=';')